In [1]:
# Import dependencies
import pandas as pd

In [2]:
# Read data
red_zone_line_player_proximity = pd.read_csv('https://storage.googleapis.com/big-data-bowl/redZoneLinePlayerProximity.csv')
red_zone_qb_proximity = pd.read_csv('https://storage.googleapis.com/big-data-bowl/redZoneQBProximity.csv')

In [ ]:
# Define matchup threshold
matchup_distance_threshold = 0
matchup_duration_threshold = 0

In [4]:
# Get unique matchups that fall below threshold for frames exceeding the matchup duration_threshold

In [ ]:
# Merge defensive player's distance to QB

In [ ]:
# Find pocket progression across matchup frames

In [ ]:
# Clean for ML model